In [8]:
from flask import Flask
import os
import webbrowser



In [11]:
from JTBrix.screen_config import flow_config


In [ ]:
flow_config.update(config)

In [9]:
from JTBrix import screen_config
screen_config.flow_config = config  # 👈 MAKE IT AVAILABLE TO OTHER MODULES
from JTBrix.experiment.run_session import run_entire_test_config
from JTBrix.screen_config import flow_config # 👈 ADD THIS LINE






ImportError: cannot import name 'screen_config' from partially initialized module 'JTBrix' (most likely due to a circular import) (/Users/amid/GitHub/JTBrix/src/JTBrix/__init__.py)

In [6]:
from JTBrix.experiment.run_session import run_entire_test_config

ImportError: cannot import name 'screen_config' from partially initialized module 'JTBrix' (most likely due to a circular import) (/Users/amid/GitHub/JTBrix/src/JTBrix/__init__.py)